**Hate speech and toxic comments detection**
--------

The goal of this TP is to train a Deep Learning model to detect Hate speech and toxic comments on social media.

The datasets provided is called `jibes-and-delights`. You can read the paper if you want: https://aclanthology.org/2021.woah-1.14.pdf

This dataset consists of 68,159 insults and 51,102 compliments sourced from the Reddit forums "RoastMe" and "ToastMe," targeting individuals rather than communities or races, and is used to benchmark multiple state-of-the-art models for classification and unsupervised style transfer.


**The goal of this practical**


The TP can be divided into 2 main part:

- 1) Build and compare Transformers models
- 2) Build recurrent models

1) First part

You need to do the following steps:

- Apply the preprocessing / data augmentation techniques you want (for data augmentation you can have a look at nlpaug library)
- Build Transformers models, compare at least 3 models, for instance:
    - DistilBERT
    - RoBERTa
    - XLMRoberta
    - flan-T5-small
- Train the models on the dataset
- Evaluate the performance of the models

IMPORTANT links:

https://huggingface.co/docs/transformers/training

2) Second part

- Apply the same steps
- Build a recurrent model
    - Bidirectional GRU

You can find the dataset here: https://drive.google.com/drive/folders/1v86GPF2N8QGZEapAo1UYOiN9UQ2omFGy?usp=sharing

In [ ]:
# Some libaries you can install

!pip install datasets
!pip install sentencepiece
!pip install accelerate -U
!pip install evaluate

In [2]:
!nvidia-smi

Wed Apr 17 14:44:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import torch
import pandas as pd
from datasets import Dataset

You can use the following functions to load the dataset


In [4]:
def file_to_dataframe(path, label):
    with open(path, 'r+') as f:
        lines = f.readlines()

    labels = [label] * len(lines)
    return pd.DataFrame({'text': lines, 'label': labels})

In [6]:
path_data = 'drive/MyDrive/hate-speech-research/datasets/jibes-and-delights/experiment-split/'

df_train_label0 = file_to_dataframe(path_data + 'comment.train.0', 0)
df_train_label1 = file_to_dataframe(path_data + 'comment.train.1', 1)

df_test_label0 = file_to_dataframe(path_data + 'comment.test.0', 0)
df_test_label1 = file_to_dataframe(path_data + 'comment.test.1', 1)

In [7]:
df_train = pd.concat([df_train_label0, df_train_label1])
df_test = pd.concat([df_test_label0, df_test_label1])

Build huggingface dataset

In [8]:
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

In [9]:
train_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 104594
})

Have a look at some samples

In [12]:
train_dataset[101]

{'text': 'It looks like you dipped your chin in honey and rubbed your face around the edge of a urinal.\n',
 'label': 0,
 '__index_level_0__': 101}

Load some pretrained model with their tokenizer

In [13]:
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer

In [14]:
# Load the model / tokenizer

# xlm roberta

# Load the tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

# Load the model
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Create a small function tokenize an input sample using padding, truncation and the max length of your model

In [15]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

Use huggingface dataset map function to tokenize the whole dataset

In [25]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset = train_dataset.shuffle(seed=42).select(range(20_000))

Map:   0%|          | 0/70000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2002 [00:00<?, ? examples/s]

In [26]:
train_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 20000
})

Use `TrainingArguments` and `Trainer` from huggingface library to finetune your model

In [27]:
from transformers import TrainingArguments, Trainer

In [28]:
# https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.TrainingArguments

training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="steps",
    eval_steps=100,
    num_train_epochs=1,
    lr_scheduler_type="linear",
    per_device_train_batch_size=16
)

Create the metrics, check the documentation of `evaluate`

In [29]:
import evaluate

In [30]:
metric = evaluate.combine(["accuracy", "f1", "precision", "recall"])
metric

In [31]:
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [32]:
# https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,No log,0.236064,0.925574,0.922355,0.964052,0.884116


[OPT] Save and load your model on huggingface hub

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()


In [ ]:
model.save_pretrained('hate_simple_XLMRobertaClassification/')

In [ ]:
# https://huggingface.co/docs/transformers/model_sharing

# Save model to hub
model.push_to_hub('hateSimpleXLMRobertaClassification')
tokenizer.push_to_hub('hateSimpleXLMRobertaClassification')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/guigux/hateSimpleXLMRobertaClassification/commit/7b9cf81d3e283e7fe605c5891ae7ad17f21b1a0d', commit_message='Upload tokenizer', commit_description='', oid='7b9cf81d3e283e7fe605c5891ae7ad17f21b1a0d', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = model.to(device)

In [ ]:
for batch in test_dataset:
    print(torch.tensor(batch['input_ids']).shape)
    print(batch['label'])
    break

torch.Size([512])
0


Create a small function to evaluate your model based on the dataset or an inut sentence

In [ ]:
def evaluate_model(model, tokenizer, dataset):
    sentences, predictions, labels = [], [], []

    for batch in dataset:
        input_ids = torch.tensor(batch['input_ids']).reshape((1, -1)).to(device)
        attention_mask = torch.tensor(batch['attention_mask']).reshape((1, -1)).to(device)
        sentences.append(batch['text'])
        label = batch['label']

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            proba = torch.sigmoid(outputs.logits)[:, 1].item()
            predictions.append(proba)
            labels.append(label)

    return sentences, predictions, labels

In [ ]:
sentences, predictions, labels = evaluate_model(model, tokenizer, test_dataset)

Display some prediction, display:
- The sentence
- The prediction
- The label

In [ ]:
for i in range(len(sentences)):
    print(sentences[i])
    print("{} -- {}\n".format(predictions[i], labels[i]))
    if i == 20:
        break

50 Shades of Grey....describes your skin, Jesus Christ you look dead

0.019176162779331207 -- 0

90% of your weight is below the frame of the picture.

0.0829528197646141 -- 0

A blind man could read a whole novel just by touching your face.

0.0769909918308258 -- 0

A blind pilot could land between your brows.

0.032674822956323624 -- 0

A chemo patient could grow a better beard

0.9765536785125732 -- 0

a goat has a better kept beard than yours

0.07794759422540665 -- 0

A hairdo doesn't make you good looking, sorry man.

0.019396934658288956 -- 0

A joint is the only smoking hot thing you'll ever have in your hands or mouth.

0.02265303023159504 -- 0

A pre-pubescent female skin cancer survivor with third degree burns could grow a better beard than you.

0.06149395555257797 -- 0

A serial killer would have to remove a shelf from his fridge to shove your tall ass head in there.

0.05162705108523369 -- 0

A slapped ass has more enthusiasm than your sorry face.

0.06527554988861084 -- 

Display the metrics:
- ROC AUC
- Accuracy
- Precision
- Recall
- F1 score

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

binary_predictions = (torch.tensor(predictions) > 0.5).int()

# Compute the performances
roc_auc = roc_auc_score(labels, predictions)
accuracy = accuracy_score(labels, binary_predictions)
f1 = f1_score(labels, binary_predictions)
precision = precision_score(labels, binary_predictions)
recall = recall_score(labels, binary_predictions)

print(roc_auc)
print(accuracy)
print(f1)
print(precision)
print(recall)

0.9876726669933463
0.9460539460539461
0.9466929911154985
0.935609756097561
0.958041958041958


Now test with manual data

In [ ]:
# TEST with manual data
sentence = "little slut"

batch = tokenizer(sentence, return_tensors='pt')
input_ids = batch['input_ids'].reshape((1, -1)).to(device)
attention_mask = batch['attention_mask'].reshape((1, -1)).to(device)

logit = model(input_ids=input_ids, attention_mask=attention_mask)
logit = logit.logits[:, 1]
proba = torch.sigmoid(logit)
print(proba)

tensor([0.9628], device='cuda:0', grad_fn=<SigmoidBackward0>)
